In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [2]:
# 크롬 웹드라이버 실행 경로
path = 'chromedriver.exe'

# 크롤링할 주소 (네이버 세부설정까지 완료한 상태로)
target_url = "https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2005.01.01&de=2005.01.15&docid=&related=0&mynews=1&office_type=1&office_section_code=3&news_office_checked=1009&nso=so%3Ar%2Cp%3Afrom20050101to20050115&is_sug_officeid=0&office_category=0&service_area=0"

# 크롬 드라이버 사용
service = Service(executable_path=path) # selenium 최근 버전은 이렇게
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

# 드라이버에게 크롤링할 대상 알려주기
try:
    # 웹페이지 열기
    driver.get(target_url)

    # 초기 스크롤 높이 설정
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 아래까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 적응 시간
        time.sleep(0.5)  

        # 새로운 스크롤 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 만약 더 스크롤할게 없다면 정지
        last_height = new_height

    # 해당 화면 안에서 페이지 소스 받기
    page_source = driver.page_source

    # BeautifulSoup으로 parsing
    bs_obj = BeautifulSoup(page_source, 'html.parser')

    # a태그로 info에 해당하는 태그들 다 수집
    results = bs_obj.find_all('a', {'class': "info"})

    # 수집한 a 태그 개수
    print(f"Number of results found: {len(results)}")

finally:
    # 가상 화면 닫기
    driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.100)
Stacktrace:
	GetHandleVerifier [0x00007FF6F8559632+30946]
	(No symbol) [0x00007FF6F850E3C9]
	(No symbol) [0x00007FF6F8406FDA]
	(No symbol) [0x00007FF6F83DCB85]
	(No symbol) [0x00007FF6F84837A7]
	(No symbol) [0x00007FF6F849A771]
	(No symbol) [0x00007FF6F847C813]
	(No symbol) [0x00007FF6F844A6E5]
	(No symbol) [0x00007FF6F844B021]
	GetHandleVerifier [0x00007FF6F868F83D+1301229]
	GetHandleVerifier [0x00007FF6F869BDB7+1351783]
	GetHandleVerifier [0x00007FF6F8692A03+1313971]
	GetHandleVerifier [0x00007FF6F858DD06+245686]
	(No symbol) [0x00007FF6F851758F]
	(No symbol) [0x00007FF6F8513804]
	(No symbol) [0x00007FF6F8513992]
	(No symbol) [0x00007FF6F850A3EF]
	BaseThreadInitThunk [0x00007FFCE219257D+29]
	RtlUserThreadStart [0x00007FFCE3DEAF28+40]


In [3]:
# 기존 한국경제 링크랑 섞여서 네이버 뉴스꺼만 필터링
news_link = []
for tag in results:
    href = tag.get('href')
    if href:
        news_link.append(href)

news_link = [i for index, i in enumerate(news_link) if (index + 1) % 2 == 0]

In [4]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [5]:
all_results = dict()
i = 0
for link in news_link:     
        # scraping하려는 웹페이지 주소를 get()에 전달
        driver.get(link)
        all_results[i] = dict()
        
        # 일시 추출하기
        date = driver.find_elements(By.CLASS_NAME, 'media_end_head_info_datestamp_bunch')
        date = date[0].text[2:]
        
        # 제목 추출하기
        title = driver.find_elements(By.CLASS_NAME, 'media_end_head_headline')
        title = title[0].text
        
        # 본문 추출하기
        body = driver.find_elements(By.ID, 'newsct_article')
        body = body[0].text.replace('\n','')
        
        
        all_results[i]['link'] = link
        all_results[i]['date'] = date
        all_results[i]['title'] = title
        all_results[i]['body'] = body
        

        i += 1
        

In [6]:
news_df = pd.DataFrame(all_results).T

In [7]:
news_df

,link,date,title,body
0,https://n.news.naver.com/mnews/article/009/000...,2005.01.03. 오전 9:38,[국내경제 전망] 주력품목 수출전선 '비상등',올해 국내 산업 전망은 그 어느 때보다 어둡다. 지난해 경제를 지탱했던 주력 산업의...
1,https://n.news.naver.com/mnews/article/009/000...,2005.01.03. 오전 9:38,"[초일류를 향해] 삼성, 李회장 ""어려울수록 투자 늘려라""",국내 최대 그룹인 삼성은 올 한 해를 성장 잠재력을 키우는 해로 삼을 태세다.200...
2,https://n.news.naver.com/mnews/article/009/000...,2005.01.03. 오전 9:38,[초일류를 향해] 대기업 '공격경영' 라인업,위기를 기회로 만들어야 한다는 생각은 언제나 경영자 머릿속을 떠나지 않는다 . 그러...
3,https://n.news.naver.com/mnews/article/009/000...,2005.01.03. 오전 9:38,日 LCD연합전선 결성,"한국 삼성전자와 LG필립스LCD, 일본 샤프 등 세계 LCD 3강을 추격하는 연합전..."
4,https://n.news.naver.com/mnews/article/009/000...,2005.01.03. 오전 9:38,"""DVD로 로열티 받는 날 온다""",DVD 다음 단계인 차세대 광(光)저장장치 세계표준을 놓고 '블루레이(Blu-Ray...
...,...,...,...,...
162,https://n.news.naver.com/mnews/article/009/000...,2005.01.14. 오후 5:11,[표] 거래소 외국인 순매수ㆍ순매도 상위종목,(단위＝억원ㆍ만주) ==============================...
163,https://n.news.naver.com/mnews/article/009/000...,2005.01.14. 오후 5:17,삼성전자 '100억弗 클럽' 가입,삼성전자가 국내 기업으로는 사상 처음으로 순이익 '100억달러 클럽' 에 가입 했다...
164,https://n.news.naver.com/mnews/article/009/000...,2005.01.14. 오후 5:17,주가 900 넘었다,14일 '양호한 실적' 을 발표한 삼성전자가 6.19% 급등한 데 힘입어 종합주가지...
165,https://n.news.naver.com/mnews/article/009/000...,2005.01.14. 오후 5:32,회사가 직원 뒤까지 챙겨?,'직원들 뒤(?)도 챙겨줘야 좋은 회사.'집보다 더 많은 시간을 보내는 회사와 기관...
